In [3]:
import numpy as np

### Introduction

The following code will allow you to compute the Floquet exponents $\mu_1(\xi_1,\xi_2)$ and $\mu_2(\xi_1,\xi_2)$ of a 2-D dynamical system

$$\mathbf{\dot{x}} = \mathcal{H}(t,\xi_1,\xi_2)\,\mathbf{x}$$

where $\mathcal{H}(t)$ is a $T$-periodic matrix depending on parameters $(\xi_1,\xi_2)$.

### User Input: RHS of ODEs

In the following block, simply enter the expressions $f_1(t,\mathbf{x},\xi_1,\xi_2)$ and $f_2(t,\mathbf{x},\xi_1,\xi_2)$ corresponding to the right hand sides of your ODEs.



In [25]:
def func1(t,x1,x2,xi1,xi2):
    return # Enter here

def func2(t,x1,x2,xi1,xi2):
    return # Enter here

### User Input: ODE Integration Parameters

Enter your desired stepside and the time interval to integrate the ODEs over.

In [22]:
h = 0.01
timespan = 1

### User Input: $(\xi_1,\xi_2)$ Range

Enter the parameter range you wish to consider.

In [4]:
start = -1
end = 1
step = 0.1
paramvals_ = np.arange(start, end, step)

### Function Blocks

In [26]:
def Solve_RK4(initval_1, initval_2, timespan, h, param1, param2):
    
    ### Integrating ODEs ###
    
    N = int(timespan / h)

    t_ = np.ones(N)
    sol1_ = np.ones(N, dtype=complex)
    sol2_ = np.ones(N, dtype=complex)

    t = 0
    sol1 = initval_1
    sol2 = initval_2

    t_[0] = t
    sol1_[0] = sol1
    sol2_[0] = sol2
    
    for i in range(1, N):
        
        K1 = h*func1(t, sol1, sol2, param1, param2) 
        L1 = h*func2(t, sol1, sol2, param1, param2) 

        K2 = h*func1(t + 0.5*h, sol1 + 0.5*K1, sol2 + 0.5*L1, param1, param2)
        L2 = h*func2(t + 0.5*h, sol1 + 0.5*K1, sol2 + 0.5*L1, param1, param2)

        K3 = h*func1(t + 0.5*h, sol1 + 0.5*K2, sol2 + 0.5*L2, param1, param2)
        L3 = h*func2(t + 0.5*h, sol1 + 0.5*K2, sol2 + 0.5*L2, param1, param2)

        K4 = h*func1(t + h, sol1 + K3, sol2 + L3, param1, param2)
        L4 = h*func2(t + h, sol1 + K3, sol2 + L3, param1, param2)

        t =  i*h
        sol1 += (K1 + 2*K2 + 2*K3 + K4)/6
        sol2 += (L1 + 2*L2 + 2*L3 + L4)/6
                
        t_[i] = t
        sol1_[i] = sol1
        sol2_[i] = sol2

    ### Getting initial and final solution vectors ###

    sol0 = np.array([sol1_[0], sol2_[0]])
    solT = np.array([sol1_[-1], sol2_[-1]])
                    
    return [t_, sol1_, sol2_, sol0, solT]

In [39]:
def Floquet_Finder_2D(timespan, h, paramvals_):

    start = time.time()

    exp1_ = np.ones((len(zetavals_),len(zetavals_)), dtype=np.ndarray)
    exp2_ = np.ones((len(zetavals_),len(zetavals_)), dtype=np.ndarray)

    ### Integrating ODEs for the initial conditions [1,0] and [0,1] ###
    ### Then, the monodromy matrix M = X(T) since X(0) = Id ###

    for i, param1 in enumerate(paramvals_):
        for j, param2 in enumerate(paramvals_):

            results_1 = Solve_RK4(1, 0, timespan, h, param1, param2)
            results_2 = Solve_RK4(0, 1, timespan, h, param1, param2)
            
            ### Constrcting monodromy matrix and computing Floquet exponents ###

            x1T, x2T = results_1[5], results_2[5]
            M = np.array([x1T, x2T])
            M = XT.transpose()
            
            multipliers, eigvecs = np.linalg.eig(M)
            exponents = np.array([phase(mult) for mult in multipliers]) # Sort exponents according to sign (positive, negative)
            
            exp1_[i,j] = (param1, param2, exponents[0])
            exp2_[i,j] = (param1, param2, exponents[1])

    end = time.time()
    
    print(end - start)
    
    return exp1_.ravel(), exp2_.ravel()

### Execution Block

In [ ]:
exponent1_, exponent2_ = Floquet_Finder_2D(timespan, h, paramvals_)